In [22]:
# derive_labels_from_unzipped.py
# Requirements: Python 3.7+, pandas (optional)
import os
from pathlib import Path
import csv

IN_PATH = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_expression.tsv"  # your unzipped file
OUT_CSV = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_with_labels.csv"

def normalize_barcode(b):
    return b.replace('.', '-')

def barcode_to_patient_id(barcode):
    parts = barcode.split('-')
    if len(parts) < 3:
        return barcode
    return '-'.join(parts[:3])

def is_normal_from_barcode(barcode):
    parts = barcode.split('-')
    if len(parts) < 4:
        return False
    code = parts[3][:2]
    return code == '11'

def read_header_samples(path):
    # Read only the first line to get header
    with open(path, 'rt', errors='replace') as f:
        header = f.readline().rstrip("\n\r")
    cols = header.split("\t")
    # first column is row identifier; the rest are samples
    return cols[1:]

def main():
    path = Path(IN_PATH)
    if not path.exists():
        raise FileNotFoundError(f"Expression file not found: {path}")

    sample_ids = read_header_samples(path)
    print(f"Found {len(sample_ids)} sample columns in header.")

    rows = []
    tumor_count = 0
    normal_count = 0
    for sid in sample_ids:
        sid_norm = normalize_barcode(sid)
        patient = barcode_to_patient_id(sid_norm)
        binary = 0 if is_normal_from_barcode(sid_norm) else 1
        if binary == 1:
            tumor_count += 1
        else:
            normal_count += 1
        rows.append((sid, sid_norm, patient, binary))

    # Write CSV
    with open(OUT_CSV, 'w', newline='') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(["original_sample_id", "normalized_sample_id", "patient_id", "binary_label"])
        for r in rows:
            w.writerow(r)

    print(f"Saved labels to: {OUT_CSV}")
    print(f"Tumor samples: {tumor_count}, Normal samples: {normal_count}")

if __name__ == "__main__":
    main()

Found 10459 sample columns in header.
Saved labels to: /Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_with_labels.csv
Tumor samples: 9732, Normal samples: 727


In [24]:
import pandas as pd
import requests
import json
import time
from tqdm import tqdm

# Input / output paths
IN_CSV = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_with_labels.csv"     # your existing labels file
OUT_CSV = "/Users/arujatiwary/Documents/local_tcga/tcga_sample_labels_with_cancer_type.csv"
BATCH_SIZE = 100

# Load your existing labels
df = pd.read_csv(IN_CSV)
patient_ids = df["patient_id"].unique().tolist()
print(f"Unique patient IDs: {len(patient_ids)}")

def query_gdc_for_projects(batch):
    """Query GDC Cases endpoint for project info"""
    url = "https://api.gdc.cancer.gov/cases"
    filters = {
        "op": "in",
        "content": {
            "field": "cases.submitter_id",
            "value": batch
        }
    }
    params = {
        "filters": json.dumps(filters),
        "fields": "submitter_id,project.project_id",
        "format": "JSON",
        "size": len(batch)
    }
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()
    hits = data.get("data", {}).get("hits", [])
    mapping = {}
    for h in hits:
        sid = h.get("submitter_id")
        proj = None
        proj_obj = h.get("project")
        if isinstance(proj_obj, dict):
            proj = proj_obj.get("project_id")
        elif isinstance(proj_obj, list) and len(proj_obj) > 0:
            proj = proj_obj[0].get("project_id")
        mapping[sid] = proj
    return mapping

# Query GDC in batches
patient_to_project = {}
for i in tqdm(range(0, len(patient_ids), BATCH_SIZE), desc="Querying GDC"):
    batch = patient_ids[i:i+BATCH_SIZE]
    try:
        mapping = query_gdc_for_projects(batch)
        patient_to_project.update(mapping)
    except Exception as e:
        print(f"Error querying batch {i}: {e}")
        for pid in batch:
            patient_to_project[pid] = None
    time.sleep(0.2)

# Map projects back into dataframe
df["project_id"] = df["patient_id"].map(patient_to_project)
missing = df["project_id"].isna().sum()
print(f"Missing project for {missing} samples.")

# Save updated file
df.to_csv(OUT_CSV, index=False)
print(f"Saved updated file with cancer type labels → {OUT_CSV}")
print(df.head())

Unique patient IDs: 9706


Querying GDC: 100%|██████████| 98/98 [02:59<00:00,  1.83s/it]

Missing project for 5 samples.
Saved updated file with cancer type labels → /Users/arujatiwary/Documents/local_tcga/tcga_sample_labels_with_cancer_type.csv
  original_sample_id normalized_sample_id    patient_id  binary_label  \
0    TCGA-S9-A7J2-01      TCGA-S9-A7J2-01  TCGA-S9-A7J2             1   
1    TCGA-G3-A3CH-11      TCGA-G3-A3CH-11  TCGA-G3-A3CH             0   
2    TCGA-EK-A2RE-01      TCGA-EK-A2RE-01  TCGA-EK-A2RE             1   
3    TCGA-44-6778-01      TCGA-44-6778-01  TCGA-44-6778             1   
4    TCGA-VM-A8C8-01      TCGA-VM-A8C8-01  TCGA-VM-A8C8             1   

  project_id  
0   TCGA-LGG  
1  TCGA-LIHC  
2  TCGA-CESC  
3  TCGA-LUAD  
4   TCGA-LGG  


In [1]:
import pandas as pd
from tqdm import tqdm
import csv

# ---------- CONFIG ----------
EXON_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/TCGA.PANCAN.sampleMap_HiSeqV2_exon"   # original dataset
LABELS_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_sample_labels_with_cancer_type.csv"                    # your labels file
OUT_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_with_labels.tsv"                                     # output merged file
# ----------------------------

def read_header(path):
    with open(path, "r", errors="replace") as f:
        header = f.readline().strip()
    cols = header.split("\t")
    sample_ids = cols[1:]
    return header, sample_ids

def main():
    print("Reading dataset header...")
    header, sample_ids = read_header(EXON_FILE)
    print(f"Found {len(sample_ids)} samples.")

    print("Loading label file...")
    labels = pd.read_csv(LABELS_FILE, dtype=str)
    labels["normalized_sample_id"] = labels["normalized_sample_id"].str.replace(".", "-")

    # Build lookup dictionaries
    binary_dict = dict(zip(labels["normalized_sample_id"], labels["binary_label"]))
    cancer_dict = dict(zip(labels["normalized_sample_id"], labels["project_id"]))

    print("Reordering labels to match dataset sample order...")
    binary_row = ["binary_label"]
    cancer_row = ["project_id"]

    missing = 0
    for sid in sample_ids:
        sid_norm = sid.replace(".", "-")
        b = binary_dict.get(sid_norm, "")
        c = cancer_dict.get(sid_norm, "")
        if b == "" or c == "":
            missing += 1
        binary_row.append(b)
        cancer_row.append(c)

    print(f"Labels missing for {missing} samples (they’ll be blank).")

    # Now copy the original dataset and append two rows
    print("Appending label rows to output file...")
    with open(OUT_FILE, "w", newline="", errors="replace") as out_f, open(EXON_FILE, "r", errors="replace") as in_f:
        # Copy the original file
        for line in tqdm(in_f, desc="Copying original dataset"):
            out_f.write(line)
        # Append our two new rows
        writer = csv.writer(out_f, delimiter="\t")
        writer.writerow(binary_row)
        writer.writerow(cancer_row)

    print(f"\nDone! File saved as: {OUT_FILE}")
    print("You can now transpose and process later as needed.")

if __name__ == "__main__":
    main()

Reading dataset header...
Found 10459 samples.
Loading label file...
Reordering labels to match dataset sample order...
Labels missing for 0 samples (they’ll be blank).
Appending label rows to output file...


Copying original dataset: 239323it [01:47, 2229.13it/s]



Done! File saved as: /Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_with_labels.tsv
You can now transpose and process later as needed.


In [7]:
import pandas as pd

path = "/Users/arujatiwary/Desktop/DL+IS-2/subsets/subset_1.tsv"
df = pd.read_csv(path, sep="\t", nrows=10)
print(df.head(10))

                            id  TCGA-EK-A2RE-01  TCGA-C8-A1HL-01  \
0     chr3:52007981-52008646:-           0.8145           1.4589   
1   chr1:215901372-215901726:-           0.0000           0.0000   
2    chr17:49268962-49270377:-           0.5150           0.7109   
3   chr4:147214081-147214132:-           2.2705           1.3928   
4     chr1:16074041-16074475:+           0.0000           0.8644   
5   chr3:195599147-195599341:-           3.6112           3.0232   
6    chr17:30351730-30351801:+           1.0323           1.6862   
7  chr14:106521424-106521433:-           0.0000           0.0000   
8    chr15:42032104-42032401:+           1.2345           0.8384   
9     chr1:94485138-94485315:-           0.0846           0.0000   

   TCGA-R6-A8WC-01  TCGA-L5-A4OH-01  TCGA-A2-A3XX-01  TCGA-DQ-5624-01  \
0           2.3137           1.0703           2.0839           1.2306   
1           0.0000           0.0000           0.0000           0.0000   
2           1.5549           1.2

In [3]:
#!/usr/bin/env python3
"""
collapse_exons_to_genes_resumable.py

Resumable, batch-aggregation pipeline:
1) Stream exon TSV as a view (no full in-RAM load)
2) UNPIVOT -> exons_long (materialized)
3) Batch-aggregate exon -> gene means into gene_expr_long_accum
   (saves checkpoint parquet during the loop)
4) Build gene_matrix (samples x genes) from accumulated table
5) Flatten and write final gene CSV

Designed for 8GB RAM + ~35GB free disk. Safe to re-run; it will resume where it left off.
"""

import duckdb, pandas as pd, os, time, math
from textwrap import dedent

# ---------- CONFIG ----------
EXON_TSV   = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_exon_expression.tsv"   # big input
PROBE_MAP  = "/Users/arujatiwary/Desktop/DL+IS-2/unc_v2_exon_hg19_probe_TCGA"
DB_PATH    = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_collapse.duckdb"
OUT_CSV    = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression.csv"
TEMP_DIR   = "/Users/arujatiwary/duckdb_temp"   # must exist or will be created
MAX_TEMP_GB = 18                                # keep below free disk (you had ~35GB free)
MEM_LIMIT  = '4GB'                              # safe for 8GB RAM
THREADS    = 1
BATCH_SIZE = 1000                               # number of genes per batch (tune if needed)
CHECKPOINT_FREQ = 5                             # save accum parquet every N batches
# -----------------------------

os.makedirs(TEMP_DIR, exist_ok=True)

def table_exists(con, name):
    try:
        rows = con.execute(f"PRAGMA table_info('{name}')").fetchall()
        return len(rows) > 0
    except Exception:
        return False

def run():
    t0 = time.time()
    print("🚀 Starting resumable collapse exons -> genes")
    print(f"Temp dir: {TEMP_DIR}, max temp {MAX_TEMP_GB} GB, memory limit {MEM_LIMIT}, threads {THREADS}")
    if os.path.exists(DB_PATH):
        # keep DB (we will re-use it) — do not remove so we can resume between runs
        pass

    # Connect and set PRAGMAs immediately (before any other duckdb actions)
    con = duckdb.connect(DB_PATH)
    con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")
    con.execute(f"PRAGMA max_temp_directory_size='{MAX_TEMP_GB}GB';")
    con.execute(f"SET memory_limit='{MEM_LIMIT}';")
    con.execute(f"SET threads={THREADS};")
    con.execute("SET preserve_insertion_order=false;")
    print("DuckDB configured with PRAGMAs.")

    # Load probe map into DuckDB (small)
    if not table_exists(con, "probe_map"):
        print("Loading probe map (this is small)...")
        probe_df = pd.read_csv(PROBE_MAP, sep="\t", dtype=str, low_memory=False)
        # normalize columns: first col -> id, second -> gene
        probe_df = probe_df.rename(columns={probe_df.columns[0]: "id", probe_df.columns[1]: "gene"})
        probe_df = probe_df.dropna(subset=["id", "gene"]).drop_duplicates()
        con.register("probe_map", probe_df)  # register in current session only
        con.execute("CREATE OR REPLACE TABLE probe_map AS SELECT * FROM probe_map;")
        print("Probe map table created in DuckDB.")
    else:
        print("probe_map table already exists in DB; skipping loading.")

    # 1) Create view exons_raw if not present
    if not table_exists(con, "exons_raw"):
        print("Registering exon TSV as streamed view (no full load)...")
        con.execute(dedent(f"""
            CREATE OR REPLACE VIEW exons_raw AS
            SELECT * FROM read_csv(
                '{EXON_TSV}',
                delim='\\t',
                header=TRUE,
                AUTO_DETECT=TRUE,
                SAMPLE_SIZE=10000,
                ALL_VARCHAR=TRUE
            );
        """))
        print("exons_raw view created.")
    else:
        print("exons_raw already exists; skipping view creation.")

    # 2) detect id column name
    cols = con.execute("PRAGMA table_info('exons_raw')").fetchall()
    if not cols:
        raise RuntimeError("exons_raw has no columns; confirm EXON_TSV path and header.")
    id_col = cols[0][1]
    print(f"Detected exon ID column: {id_col}")

    # 3) create exons_long if not exists
    if not table_exists(con, "exons_long"):
        print("Creating exons_long (UNPIVOT)... This will materialize exons_long table in DB.")
        con.execute(dedent(f"""
            CREATE OR REPLACE TABLE exons_long AS
            SELECT *
            FROM exons_raw
            UNPIVOT (expression FOR sample_id IN (COLUMNS(* EXCLUDE "{id_col}")));
        """))
        print("exons_long created.")
        # checkpoint exons_long (optional parquet) - can be big, but useful for resume
        ex_long_parquet = os.path.join(TEMP_DIR, "exons_long.parquet")
        print(f"Writing checkpoint exons_long -> {ex_long_parquet} (parquet) ...")
        con.execute(f"COPY (SELECT * FROM exons_long) TO '{ex_long_parquet}' (FORMAT PARQUET);")
        print("Checkpoint exons_long saved.")
    else:
        print("exons_long already exists; skipping UNPIVOT. (If you want to recompute, delete exons_long table first.)")

    # 4) Prepare gene list from probe_map table (unique genes)
    genes = [r[0] for r in con.execute("SELECT DISTINCT gene FROM probe_map WHERE gene IS NOT NULL ORDER BY gene").fetchall()]
    print(f"Total unique genes from probe map: {len(genes)}")

    # 5) create accumulator table if not exists
    if not table_exists(con, "gene_expr_long_accum"):
        con.execute("CREATE TABLE gene_expr_long_accum(gene_symbol VARCHAR, sample_id VARCHAR, expr_mean DOUBLE);")
        print("Created accumulator table gene_expr_long_accum.")
    else:
        existing = con.execute("SELECT COUNT(*) FROM gene_expr_long_accum").fetchone()[0]
        print(f"gene_expr_long_accum already exists with {existing} rows; will append missing genes.")

    # 6) Batch-aggregate genes -> append into gene_expr_long_accum
    total_genes = len(genes)
    batches = math.ceil(total_genes / BATCH_SIZE)
    print(f"Aggregating in batches: {BATCH_SIZE} genes per batch -> {batches} batches total.")

    for i in range(0, total_genes, BATCH_SIZE):
        batch_genes = genes[i:i+BATCH_SIZE]
        # prepare SQL-safe quoted list
        batch_list = ",".join(["'" + g.replace("'", "''") + "'" for g in batch_genes])

        # check whether any genes from this batch already processed
        q = con.execute(f"SELECT COUNT(DISTINCT gene_symbol) FROM gene_expr_long_accum WHERE gene_symbol IN ({batch_list});").fetchone()[0]
        if q == len(batch_genes):
            print(f"Batch {i//BATCH_SIZE + 1}/{batches} - all genes already processed; skipping.")
            continue
        else:
            print(f"Batch {i//BATCH_SIZE + 1}/{batches} - processing genes {i+1}..{i+len(batch_genes)} (count processed in this batch so far: {q})")

        # compute batch_result for this set of genes
        # We join probe_map to exons_long using the proper id column name
        # Use a small, explicit temp table name for this batch and then insert into accum
        batch_tbl = f"batch_result_{i//BATCH_SIZE + 1}"
        # drop if exists
        if table_exists(con, batch_tbl):
            con.execute(f"DROP TABLE {batch_tbl};")

        sql = dedent(f"""
            CREATE TABLE {batch_tbl} AS
            SELECT p.gene AS gene_symbol,
                   el.sample_id AS sample_id,
                   AVG(CAST(el.expression AS DOUBLE)) AS expr_mean
            FROM exons_long el
            JOIN probe_map p
              ON el."{id_col}" = p.id
            WHERE p.gene IN ({batch_list})
            GROUP BY p.gene, el.sample_id;
        """)
        con.execute(sql)
        # append into accumulator but avoid duplicates just in case
        con.execute(f"INSERT INTO gene_expr_long_accum SELECT * FROM {batch_tbl} WHERE (gene_symbol, sample_id) NOT IN (SELECT gene_symbol, sample_id FROM gene_expr_long_accum);")
        con.execute(f"DROP TABLE {batch_tbl};")
        print(f"Batch {i//BATCH_SIZE + 1} appended to accumulator. Accum size now ~ {con.execute('SELECT COUNT(*) FROM gene_expr_long_accum').fetchone()[0]} rows")

        # checkpoint occasionally: write the accum table to parquet
        batch_no = (i // BATCH_SIZE) + 1
        if batch_no % CHECKPOINT_FREQ == 0 or batch_no == batches:
            accum_parquet = os.path.join(TEMP_DIR, f"gene_expr_long_accum_checkpoint_batch{batch_no}.parquet")
            print(f"Writing checkpoint parquet for accum -> {accum_parquet}")
            con.execute(f"COPY (SELECT * FROM gene_expr_long_accum) TO '{accum_parquet}' (FORMAT PARQUET);")
            print("Checkpoint written.")

    print("All batches processed. Final accumulator size:", con.execute("SELECT COUNT(*) FROM gene_expr_long_accum").fetchone()[0])

    # 7) Create gene_matrix (map_aggregate) from accum table (this is cheaper than full grouping)
    if not table_exists(con, "gene_matrix"):
        print("Creating gene_matrix (mapping genes -> map structure per sample)...")
        con.execute(dedent("""
            CREATE TABLE gene_matrix AS
            SELECT sample_id,
                   map_aggregate(gene_symbol, expr_mean) AS gene_map
            FROM gene_expr_long_accum
            GROUP BY sample_id;
        """))
        print("gene_matrix created.")
    else:
        print("gene_matrix already exists; skipping.")

    # 8) Flatten gene_matrix to columns
    if not table_exists(con, "gene_matrix_flat"):
        print("Flattening gene_map to wide columns (this can take time but is final)...")
        con.execute(dedent("""
            CREATE TABLE gene_matrix_flat AS
            SELECT sample_id, * EXCLUDE (sample_id)
            FROM gene_matrix, UNNEST(gene_map);
        """))
        print("gene_matrix_flat created.")
    else:
        print("gene_matrix_flat already exists; skipping flatten step.")

    # 9) Export final CSV if not exists
    if not os.path.exists(OUT_CSV):
        print(f"Writing final CSV to {OUT_CSV} (this may take a few minutes)...")
        con.execute(f"COPY (SELECT * FROM gene_matrix_flat) TO '{OUT_CSV}' (HEADER, DELIMITER ',');")
        print("Final CSV written.")
    else:
        print(f"Final CSV {OUT_CSV} already exists; skipping export.")

    # 10) summary
    rows = con.execute("SELECT COUNT(DISTINCT sample_id) FROM gene_matrix_flat").fetchone()[0]
    cols = len(con.execute("PRAGMA table_info('gene_matrix_flat')").fetchall())
    print(f"✅ Done. Samples (rows): {rows}   Genes (columns): {cols-1}")
    print("Elapsed: {:.2f} minutes".format((time.time() - t0) / 60.0))
    con.close()

if __name__ == "__main__":
    run()

🚀 Starting resumable collapse exons -> genes
Temp dir: /Users/arujatiwary/duckdb_temp, max temp 18 GB, memory limit 4GB, threads 1
DuckDB configured with PRAGMAs.
Loading probe map (this is small)...
Probe map table created in DuckDB.
exons_raw already exists; skipping view creation.
Detected exon ID column: Sample
exons_long already exists; skipping UNPIVOT. (If you want to recompute, delete exons_long table first.)
Total unique genes from probe map: 23547
Created accumulator table gene_expr_long_accum.
Aggregating in batches: 1000 genes per batch -> 24 batches total.
Batch 1/24 - processing genes 1..1000 (count processed in this batch so far: 0)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Batch 1 appended to accumulator. Accum size now ~ 10459000 rows
Batch 2/24 - processing genes 1001..2000 (count processed in this batch so far: 0)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Batch 24 appended to accumulator. Accum size now ~ 246278073 rows
Writing checkpoint parquet for accum -> /Users/arujatiwary/duckdb_temp/gene_expr_long_accum_checkpoint_batch24.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Checkpoint written.
All batches processed. Final accumulator size: 246278073
Creating gene_matrix (mapping genes -> map structure per sample)...


CatalogException: Catalog Error: Scalar Function with name map_aggregate does not exist!
Did you mean "map_entries"?

LINE 4:        map_aggregate(gene_symbol, expr_mean) AS gene_map
               ^

In [ ]:
import math
print("map_aggregate not available — pivoting in batches to build wide matrix...")

PIVOT_BATCH = 800  # safe batch size
genes_all = [r[0] for r in con.execute("SELECT DISTINCT gene_symbol FROM gene_expr_long_accum ORDER BY gene_symbol").fetchall()]
total_genes = len(genes_all)
print(f"Total genes to pivot: {total_genes}")
batches = math.ceil(total_genes / PIVOT_BATCH)
print(f"Pivoting in {batches} batches of up to {PIVOT_BATCH} genes each.")

def sql_col_name(g):
    return g.replace('"', '""').replace(" ", "_").replace("-", "_").replace("/", "_")

# create list of samples first
if not table_exists(con, "samples_list"):
    con.execute("CREATE OR REPLACE TABLE samples_list AS SELECT DISTINCT sample_id FROM gene_expr_long_accum;")
    print("Created samples_list table.")

part_tables = []
for i in range(0, total_genes, PIVOT_BATCH):
    batch_genes = genes_all[i:i+PIVOT_BATCH]
    batch_no = (i // PIVOT_BATCH) + 1
    part_name = f"gene_part_{batch_no}"
    part_tables.append(part_name)
    if table_exists(con, part_name):
        print(f"{part_name} exists — skipping.")
        continue

    print(f"Creating {part_name} for genes {i+1}..{i+len(batch_genes)}")

    # Build CASE expressions safely
    cases = []
    for g in batch_genes:
        g_esc = g.replace("'", "''")
        alias = sql_col_name(g)
        cases.append(f"MAX(CASE WHEN gene_symbol = '{g_esc}' THEN expr_mean END) AS \"{alias}\"")
    cases_sql = ",\n       ".join(cases)

    # Build a simple comma-separated list of quoted genes
    quoted_genes = ",".join("'" + g.replace("'", "''") + "'" for g in batch_genes)

    pivot_sql = f"""
    CREATE TABLE {part_name} AS
    SELECT sample_id,
           {cases_sql}
    FROM gene_expr_long_accum
    WHERE gene_symbol IN ({quoted_genes})
    GROUP BY sample_id;
    """
    con.execute(pivot_sql)
    print(f"{part_name} created.")

# ---- join all part tables into one ----
final_table = "gene_matrix_flat"
if table_exists(con, final_table):
    print(f"{final_table} already exists — skipping join step.")
else:
    print("Joining part tables into final wide matrix...")
    cur_table = "samples_list"
    for idx, part in enumerate(part_tables):
        next_table = f"join_step_{idx+1}"
        print(f"Joining {cur_table} with {part} -> {next_table}")
        con.execute(f"""
        CREATE TABLE {next_table} AS
        SELECT s.*, p.*
        FROM {cur_table} s
        LEFT JOIN {part} p
        ON s.sample_id = p.sample_id;
        """)
        if cur_table != "samples_list":
            con.execute(f"DROP TABLE IF EXISTS {cur_table};")
        cur_table = next_table

    con.execute(f"CREATE TABLE {final_table} AS SELECT * FROM {cur_table};")
    con.execute(f"DROP TABLE IF EXISTS {cur_table};")
    print(f"Final wide table {final_table} created.")

# ---- export final CSV ----2
if not os.path.exists(OUT_CSV):
    print("Writing final CSV ...")
    con.execute(f"COPY (SELECT * FROM gene_matrix_flat) TO '{OUT_CSV}' (HEADER, DELIMITER ',');")
    print("Final CSV written.")
else:
    print("Final CSV already exists; skip writing.")

print("✅ Pivot-in-batches complete.")

map_aggregate not available — pivoting in batches to build wide matrix...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total genes to pivot: 23547
Pivoting in 30 batches of up to 800 genes each.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Created samples_list table.
Creating gene_part_1 for genes 1..800


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Joining join_step_11 with gene_part_12 -> join_step_12


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Joining join_step_12 with gene_part_13 -> join_step_13


OutOfMemoryException: Out of Memory Error: could not allocate block of size 256.0 KiB (3.7 GiB/3.7 GiB used)

Possible solutions:
* Reducing the number of threads (SET threads=X)
* Disabling insertion-order preservation (SET preserve_insertion_order=false)
* Increasing the memory limit (SET memory_limit='...GB')

See also https://duckdb.org/docs/stable/guides/performance/how_to_tune_workloads

In [12]:
#!/usr/bin/env python3
"""
collapse_exons_to_genes_final_lowmem.py
--------------------------------------
Resumes from gene_expr_long_accum inside your DuckDB file and
creates the final wide gene × sample CSV using small memory-safe batches.
"""

import duckdb, os, math, time

# ---------- PATH CONFIG ----------
DB_PATH  = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_collapse.duckdb"
TEMP_DIR = "/Users/arujatiwary/duckdb_temp"
OUT_CSV  = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression.csv"
# ---------------------------------

os.makedirs(TEMP_DIR, exist_ok=True)
os.environ["DUCKDB_TEMP_DIRECTORY"] = TEMP_DIR

def table_exists(con, name):
    try:
        rows = con.execute(f"PRAGMA table_info('{name}')").fetchall()
        return len(rows) > 0
    except Exception:
        return False

print("🚀 Reconnecting to existing DuckDB database...")
con = duckdb.connect(DB_PATH)
con.execute("PRAGMA max_temp_directory_size='18GB';")
con.execute("SET memory_limit='2GB';")          # safe for 8 GB RAM
con.execute("SET threads=1;")
con.execute("SET preserve_insertion_order=false;")
print(f"✅ Connected. Temp dir: {TEMP_DIR}")

# Optional: free space by dropping old large tables
if table_exists(con, "exons_long"):
    try:
        con.execute("DROP TABLE IF EXISTS exons_long;")
        print("Dropped exons_long to reclaim space.")
    except Exception as e:
        print("Warning dropping exons_long:", e)

# ---- Parameters ----
PIVOT_BATCH = 200       # genes per batch; smaller = safer
print(f"Pivoting with PIVOT_BATCH={PIVOT_BATCH}")
t0 = time.time()

# ---- Gather gene list ----
genes_all = [r[0] for r in con.execute("SELECT DISTINCT gene_symbol FROM gene_expr_long_accum ORDER BY gene_symbol").fetchall()]
total_genes = len(genes_all)
batches = math.ceil(total_genes / PIVOT_BATCH)
print(f"Total genes: {total_genes}  →  {batches} batches")

# ---- Ensure sample list ----
if not table_exists(con, "samples_list"):
    con.execute("CREATE OR REPLACE TABLE samples_list AS SELECT DISTINCT sample_id FROM gene_expr_long_accum;")
    print("Created samples_list.")

# ---- Pivot batches ----
part_tables = []
for i in range(0, total_genes, PIVOT_BATCH):
    batch_genes = genes_all[i:i+PIVOT_BATCH]
    batch_no = (i // PIVOT_BATCH) + 1
    part_name = f"gene_part_{batch_no}"
    part_tables.append(part_name)

    if table_exists(con, part_name):
        print(f"{part_name} exists — skipping.")
        continue

    print(f"[Batch {batch_no}/{batches}] Creating {part_name} for genes {i+1}–{i+len(batch_genes)}")
    quoted_genes = ",".join("'" + g.replace("'", "''") + "'" for g in batch_genes)

    # build CASE expressions
    cases = []
    for g in batch_genes:
        g_esc = g.replace("'", "''")
        alias = g.replace('"', '""').replace(" ", "_").replace("-", "_").replace("/", "_")
        cases.append(f"MAX(CASE WHEN gene_symbol = '{g_esc}' THEN expr_mean END) AS \"{alias}\"")
    cases_sql = ",\n       ".join(cases)

    pivot_sql = f"""
    CREATE TABLE {part_name} AS
    SELECT sample_id,
           {cases_sql}
    FROM gene_expr_long_accum
    WHERE gene_symbol IN ({quoted_genes})
    GROUP BY sample_id;
    """
    con.execute(pivot_sql)
    print(f"✅ {part_name} created.")

print("\n✅ All part tables ready. Starting incremental join phase...\n")

# ---- Join part tables incrementally ----
final_table = "gene_matrix_flat"
if table_exists(con, final_table):
    print(f"{final_table} already exists — skipping join step.")
else:
    cur_table = "samples_list"
    total_parts = len(part_tables)

    for idx, part in enumerate(part_tables):
        next_table = f"join_step_{idx+1}"
        print(f"[{idx+1}/{total_parts}] Joining {cur_table} + {part} → {next_table}")

        con.execute(f"""
        CREATE TABLE {next_table} AS
        SELECT s.*, p.*
        FROM {cur_table} s
        LEFT JOIN {part} p
        ON s.sample_id = p.sample_id;
        """)

        # drop previous join table to free memory
        if cur_table != "samples_list":
            con.execute(f"DROP TABLE IF EXISTS {cur_table};")

        # force checkpoint to flush temp space
        con.execute("PRAGMA checkpoint;")

        cur_table = next_table
        print(f"✅ Joined {part} (step {idx+1}/{total_parts})")

    con.execute(f"CREATE TABLE {final_table} AS SELECT * FROM {cur_table};")
    con.execute(f"DROP TABLE IF EXISTS {cur_table};")
    print("✅ Final wide table gene_matrix_flat created.")

# ---- Export to CSV ----
if not os.path.exists(OUT_CSV):
    print("\n💾 Writing final CSV (this may take a few minutes)...")
    con.execute(f"COPY (SELECT * FROM gene_matrix_flat) TO '{OUT_CSV}' (HEADER, DELIMITER ',');")
    print(f"✅ Final CSV written: {OUT_CSV}")
else:
    print("Final CSV already exists; skipping write.")

# ---- Done ----
rows = con.execute("SELECT COUNT(DISTINCT sample_id) FROM gene_matrix_flat").fetchone()[0]
cols = len(con.execute("PRAGMA table_info('gene_matrix_flat')").fetchall())
con.close()
print(f"\n🎉 Done! Samples (rows): {rows}   Genes (columns): {cols-1}")
print(f"Elapsed: {(time.time()-t0)/60:.1f} min")

🚀 Reconnecting to existing DuckDB database...
✅ Connected. Temp dir: /Users/arujatiwary/duckdb_temp
Pivoting with PIVOT_BATCH=200
Total genes: 23547  →  118 batches
gene_part_1 exists — skipping.
gene_part_2 exists — skipping.
gene_part_3 exists — skipping.
gene_part_4 exists — skipping.
gene_part_5 exists — skipping.
gene_part_6 exists — skipping.
gene_part_7 exists — skipping.
gene_part_8 exists — skipping.
gene_part_9 exists — skipping.
gene_part_10 exists — skipping.
gene_part_11 exists — skipping.
gene_part_12 exists — skipping.
gene_part_13 exists — skipping.
gene_part_14 exists — skipping.
gene_part_15 exists — skipping.
gene_part_16 exists — skipping.
gene_part_17 exists — skipping.
gene_part_18 exists — skipping.
gene_part_19 exists — skipping.
gene_part_20 exists — skipping.
gene_part_21 exists — skipping.
gene_part_22 exists — skipping.
gene_part_23 exists — skipping.
gene_part_24 exists — skipping.
gene_part_25 exists — skipping.
gene_part_26 exists — skipping.
gene_part_27

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ gene_part_32 created.
[Batch 33/118] Creating gene_part_33 for genes 6401–6600


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ gene_part_33 created.
[Batch 34/118] Creating gene_part_34 for genes 6601–6800


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ gene_part_117 created.
[Batch 118/118] Creating gene_part_118 for genes 23401–23547


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ gene_part_118 created.

✅ All part tables ready. Starting incremental join phase...

[1/118] Joining samples_list + gene_part_1 → join_step_1


CatalogException: Catalog Error: Pragma Function with name checkpoint does not exist, but a table function with the same name exists, try `CALL checkpoint(...)`

In [17]:
import os
import pandas as pd
import re

EXPORT_DIR = "/Users/arujatiwary/Desktop/DL+IS-2/gene_parts_csv"
FINAL_CSV = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression_final.csv"

def clean_col(c):
    # sanitize column names to avoid merge errors
    c = str(c)
    c = re.sub(r'[^0-9a-zA-Z_]', '_', c)   # replace bad chars
    c = re.sub(r'_+', '_', c)              # collapse multiple underscores
    return c.strip('_')

# Find all part CSVs
tables = sorted(
    [f for f in os.listdir(EXPORT_DIR) if f.startswith("gene_part_") and f.endswith(".csv")]
)
print(f"Found {len(tables)} gene_part CSV files.")

# Load base
samples_path = os.path.join(EXPORT_DIR, "samples_list.csv")
base = pd.read_csv(samples_path)
base.set_index("sample_id", inplace=True)
print(f"Loaded base: {base.shape}")

for i, file in enumerate(tables, 1):
    part_path = os.path.join(EXPORT_DIR, file)
    print(f"[{i}/{len(tables)}] Merging {file} ...")

    part = pd.read_csv(part_path)

    # Detect and fix sample ID column
    if "sample_id" in part.columns:
        part.set_index("sample_id", inplace=True)
    else:
        first_col = part.columns[0]
        part.rename(columns={first_col: "sample_id"}, inplace=True)
        part.set_index("sample_id", inplace=True)

    # Clean column names
    part.columns = [clean_col(c) for c in part.columns]

    # Drop duplicate columns (happens due to alias collisions)
    part = part.loc[:, ~part.columns.duplicated()]

    # Align sample IDs
    part = part.reindex(base.index)

    # Join one batch at a time, streaming-safe
    base = pd.concat([base, part], axis=1)

    del part

    # Write checkpoint every 10 merges
    if i % 10 == 0:
        chk_path = os.path.join(EXPORT_DIR, f"checkpoint_{i}.parquet")
        base.to_parquet(chk_path)
        print(f"💾 Checkpoint saved ({base.shape})")

# Final cleanup of column names again
base.columns = [clean_col(c) for c in base.columns]
base = base.loc[:, ~base.columns.duplicated()]

# Save final CSV
base.to_csv(FINAL_CSV)
print(f"\n✅ Final merged CSV written: {FINAL_CSV}")
print(f"Final shape: {base.shape}")

Found 118 gene_part CSV files.
Loaded base: (10459, 0)
[1/118] Merging gene_part_1.csv ...
[2/118] Merging gene_part_10.csv ...
[3/118] Merging gene_part_100.csv ...
[4/118] Merging gene_part_101.csv ...
[5/118] Merging gene_part_102.csv ...
[6/118] Merging gene_part_103.csv ...
[7/118] Merging gene_part_104.csv ...
[8/118] Merging gene_part_105.csv ...
[9/118] Merging gene_part_106.csv ...
[10/118] Merging gene_part_107.csv ...
💾 Checkpoint saved ((10459, 3200))
[11/118] Merging gene_part_108.csv ...
[12/118] Merging gene_part_109.csv ...
[13/118] Merging gene_part_11.csv ...
[14/118] Merging gene_part_110.csv ...
[15/118] Merging gene_part_111.csv ...
[16/118] Merging gene_part_112.csv ...
[17/118] Merging gene_part_113.csv ...
[18/118] Merging gene_part_114.csv ...
[19/118] Merging gene_part_115.csv ...
[20/118] Merging gene_part_116.csv ...
💾 Checkpoint saved ((10459, 5800))
[21/118] Merging gene_part_117.csv ...
[22/118] Merging gene_part_118.csv ...
[23/118] Merging gene_part_12.

ValueError: Duplicate column names found: ['7SK', 'A1BG', 'A1BG_CTD_2619J13_8', 'A1BG_AS1', 'A1CF', 'A2M', 'A2M_AS1', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AADAT', 'AAED1', 'AAGAB', 'AAK1', 'AAK1_RP11_427H3_3_Y_RNA', 'AAMDC', 'AAMP', 'AANAT', 'AAR2', 'AARD', 'AARS', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AASS', 'AATF', 'AATK', 'AATK_AS1', 'AB019441_29', 'ABAT', 'ABC7_42389800N19_1', 'ABCA1', 'ABCA10', 'ABCA10_ABCA5', 'ABCA12', 'ABCA13', 'ABCA17P', 'ABCA2', 'ABCA3', 'ABCA4', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA8', 'ABCA9', 'ABCB1', 'ABCB10', 'ABCB11', 'ABCB4', 'ABCB5', 'ABCB6', 'ABCB6_ATG9A', 'ABCB7_AMDP1_TRPC6P_TCEB1P24_DDX11L16_WASIR1', 'ABCB8', 'ABCB9', 'ABCC1', 'ABCC10', 'ABCC11', 'ABCC11_RP11_3M1_1', 'ABCC12', 'ABCC12_Y_RNA', 'ABCC13', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCC6', 'ABCC6P1', 'ABCC6P2', 'ABCC8', 'ABCC9', 'ABCD1_IL9R_SPRY3_VAMP7_WASH6P_DPH3P2_AJ271736_10', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCE1', 'ABCF1', 'ABCF2', 'A

In [ ]:
import os, pandas as pd

EXPORT_DIR = "/Users/arujatiwary/Desktop/DL+IS-2/gene_parts_csv"
FINAL_CSV  = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression_final.csv"

if 'base' not in globals():
    chk_files = sorted([f for f in os.listdir(EXPORT_DIR) if f.startswith("checkpoint_") and f.endswith(".parquet")])
    if chk_files:
        latest = chk_files[-1]
        print("Loading latest checkpoint:", latest)
        base = pd.read_parquet(os.path.join(EXPORT_DIR, latest))
    else:
        print("No checkpoint found in EXPORT_DIR. Loading samples_list as base.")
        base = pd.read_csv(os.path.join(EXPORT_DIR, "samples_list.csv")).set_index("sample_id")

print("Current base shape before dedupe:", base.shape)

# collapse duplicate-named columns by taking mean across duplicates
if base.columns.duplicated().any():
    print("Duplicate column names detected. Collapsing by mean...")
    base = base.groupby(base.columns, axis=1).mean()
    print("After collapsing duplicates, shape:", base.shape)
else:
    print("No duplicate column names detected; nothing to collapse.")

# Save a safe checkpoint (parquet) after deduping
safe_chk = os.path.join(EXPORT_DIR, "checkpoint_dedup.parquet")
base.to_parquet(safe_chk)
print("Wrote deduped checkpoint:", safe_chk)


if any(f.startswith("gene_part_") for f in os.listdir(EXPORT_DIR)):
    # if merging is complete, this will simply re-export final CSV
    pass

# Final write 
print("Final dedupe before CSV write (just in case)...")
base = base.groupby(base.columns, axis=1).mean()
print("Final shape:", base.shape)
base.to_csv(FINAL_CSV)
print("Final CSV written:", FINAL_CSV)

Current base shape before dedupe: (10459, 20547)
Duplicate column names detected. Collapsing by mean...


/var/folders/5y/zh95d1496_bg3syh_09zx1wh0000gn/T/ipykernel_13844/2610167499.py:23: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  base = base.groupby(base.columns, axis=1).mean()


After collapsing duplicates, shape: (10459, 17947)
Wrote deduped checkpoint: /Users/arujatiwary/Desktop/DL+IS-2/gene_parts_csv/checkpoint_dedup.parquet
Final dedupe before CSV write (just in case)...


/var/folders/5y/zh95d1496_bg3syh_09zx1wh0000gn/T/ipykernel_13844/2610167499.py:41: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  base = base.groupby(base.columns, axis=1).mean()


Final shape: (10459, 17947)
Final CSV written: /Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression_final.csv


In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# ---------- FILE PATHS ----------
EXPR_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_gene_expression_final.csv"
LABELS_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_sample_labels_with_cancer_type.csv"  # your label file
OUT_FILE = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_preprocessed.csv"
# --------------------------------

print("Loading expression data (this may take a minute)...")
expr = pd.read_csv(EXPR_FILE, low_memory=False)
print(f"Expression shape: {expr.shape}")

# If sample_id is not an index, make it one
if 'sample_id' in expr.columns:
    expr.set_index('sample_id', inplace=True)

import pandas as pd

print("Loading labels in streamed mode...")
chunks = pd.read_csv(
    LABELS_FILE,
    chunksize=5000,      # read 5k rows at a time
    low_memory=False
)
labels = pd.concat(chunks, ignore_index=True)
print(f"Labels loaded: {labels.shape}")


# Use normalized_sample_id to align
labels.set_index('normalized_sample_id', inplace=True)

# Align expression and labels
expr = expr.loc[expr.index.intersection(labels.index)]
labels = labels.loc[expr.index]
print(f"Aligned samples: {expr.shape[0]}")

# 1️⃣ Drop columns (genes) that are entirely missing or constant
print("Dropping all-NaN and zero-variance columns...")
expr = expr.dropna(axis=1, how='all')
expr = expr.loc[:, expr.nunique() > 1]
print(f"Remaining genes: {expr.shape[1]}")

# 2️⃣ Impute missing values (median per column)
print("Imputing missing values...")
expr = expr.fillna(expr.median())

# 3️⃣ Normalize each gene (Z-score)
print("Normalizing features...")
scaler = StandardScaler()
expr_scaled = pd.DataFrame(
    scaler.fit_transform(expr),
    index=expr.index,
    columns=expr.columns
)

# 4️⃣ Reattach labels
expr_scaled['binary_label'] = labels['binary_label']
expr_scaled['cancer_type'] = labels['project_id']

# 5️⃣ Save clean file
expr_scaled.to_csv(OUT_FILE)
print(f"✅ Preprocessed data saved to {OUT_FILE}")
print(f"Final shape: {expr_scaled.shape}")

# 6️⃣ Basic sanity checks
print("\nSanity check:")
print("Binary label distribution:")
print(expr_scaled['binary_label'].value_counts())
print("\nCancer type counts (top 10):")
print(expr_scaled['cancer_type'].value_counts().head(10))

Loading expression data (this may take a minute)...
Expression shape: (10459, 17948)
Loading labels in streamed mode...
Labels loaded: (10459, 5)
Aligned samples: 10459
Dropping all-NaN and zero-variance columns...
Remaining genes: 17947
Imputing missing values...
Normalizing features...
✅ Preprocessed data saved to /Users/arujatiwary/Desktop/DL+IS-2/tcga_preprocessed.csv
Final shape: (10459, 17949)

Sanity check:
Binary label distribution:
binary_label
1    9732
0     727
Name: count, dtype: int64

Cancer type counts (top 10):
cancer_type
TCGA-BRCA    1217
TCGA-KIRC     606
TCGA-LUAD     576
TCGA-THCA     572
TCGA-HNSC     566
TCGA-LUSC     552
TCGA-PRAD     550
TCGA-LGG      530
TCGA-SKCM     473
TCGA-STAD     450
Name: count, dtype: int64


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

# ---- CONFIG ----
DATA_PATH = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_preprocessed.csv"
LABEL_COLUMN = "binary_label"     # or 'cancer_type'
SAMPLE_FRACTION = 0.2
# ----------------

print("\n🧪 Loading data...")
df = pd.read_csv(DATA_PATH)
if SAMPLE_FRACTION < 1.0:
    df = df.sample(frac=SAMPLE_FRACTION, random_state=42)
print(f"Data shape after sampling: {df.shape}")

# Ensure label exists
if LABEL_COLUMN not in df.columns:
    raise ValueError(f"Label column '{LABEL_COLUMN}' not found in CSV.")

# Separate features and labels
y = df[LABEL_COLUMN]
X = df.drop(columns=["binary_label", "cancer_type"], errors="ignore")

# Keep only numeric columns
X = X.select_dtypes(include=[np.number])
print(f"Numeric features retained: {X.shape[1]}")

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Print label balance
print("\nLabel distribution (train):", Counter(y_train))
print("Label distribution (val):", Counter(y_val))

# RandomForest sanity check
print("\n🌲 Running RandomForest sanity check...")
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced" if len(np.unique(y_train)) == 2 else None
)

rf.fit(X_train_scaled, y_train)
rf_preds = rf.predict(X_val_scaled)
acc = accuracy_score(y_val, rf_preds)

print(f"✅ RandomForest Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, rf_preds))


🧪 Loading data...
Data shape after sampling: (2092, 17950)
Numeric features retained: 17947

Label distribution (train): Counter({1: 1547, 0: 126})
Label distribution (val): Counter({1: 387, 0: 32})

🌲 Running RandomForest sanity check...
✅ RandomForest Accuracy: 0.9642

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.56      0.71        32
           1       0.96      1.00      0.98       387

    accuracy                           0.96       419
   macro avg       0.96      0.78      0.84       419
weighted avg       0.96      0.96      0.96       419



In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

# ---- CONFIG ----
DATA_PATH = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_preprocessed.csv"
LABEL_COLUMN = "cancer_type"     # multiclass label
SAMPLE_FRACTION = 0.2
RANDOM_STATE = 42
# ----------------

print("\n🧪 Loading data...")
df = pd.read_csv(DATA_PATH)
if SAMPLE_FRACTION < 1.0:
    df = df.sample(frac=SAMPLE_FRACTION, random_state=RANDOM_STATE)
print(f"Data shape after sampling: {df.shape}")

# 1) Ensure label exists and drop rows with missing labels
if LABEL_COLUMN not in df.columns:
    raise ValueError(f"Label column '{LABEL_COLUMN}' not found in data.")

df = df[~df[LABEL_COLUMN].isna()].copy()
print(f"After dropping samples with missing label: {df.shape}")

# 2) Separate y and numeric X
y = df[LABEL_COLUMN].astype(str)   # ensure categorical string labels
X = df.drop(columns=["binary_label", "cancer_type"], errors="ignore")
X = X.select_dtypes(include=[np.number])   # numeric features only
print(f"Numeric features retained: {X.shape[1]}")

# 3) Impute numeric NaNs with column medians (in-place)
if X.isna().any().any():
    print("Imputing numeric NaNs with column medians...")
    medians = X.median()
    X = X.fillna(medians)
    print("Imputation done.")

# 4) Final check for NaNs
if X.isna().any().any() or y.isna().any():
    raise RuntimeError("Unexpected NaNs remain in X or y after cleaning.")

# 5) Train/val split (stratified)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
print("Label distribution (train):", Counter(y_train))
print("Label distribution (val):", Counter(y_val))

# 6) Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 7) Train RandomForest (multiclass)
print("\n🌲 Running Multiclass RandomForest sanity check...")
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
rf_preds = rf.predict(X_val_scaled)
acc = accuracy_score(y_val, rf_preds)

print(f"✅ Multiclass RandomForest Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, rf_preds, zero_division=0))


🧪 Loading data...
Data shape after sampling: (2092, 17950)
After dropping samples with missing label: (2091, 17950)
Numeric features retained: 17947
Label distribution (train): Counter({'TCGA-BRCA': 201, 'TCGA-HNSC': 106, 'TCGA-THCA': 106, 'TCGA-KIRC': 99, 'TCGA-LUSC': 93, 'TCGA-LUAD': 93, 'TCGA-LGG': 83, 'TCGA-PRAD': 78, 'TCGA-STAD': 75, 'TCGA-SKCM': 73, 'TCGA-BLCA': 67, 'TCGA-LIHC': 63, 'TCGA-KIRP': 51, 'TCGA-COAD': 50, 'TCGA-SARC': 42, 'TCGA-CESC': 41, 'TCGA-OV': 40, 'TCGA-UCEC': 36, 'TCGA-ESCA': 33, 'TCGA-PCPG': 28, 'TCGA-PAAD': 26, 'TCGA-GBM': 25, 'TCGA-TGCT': 25, 'TCGA-LAML': 23, 'TCGA-THYM': 20, 'TCGA-READ': 15, 'TCGA-KICH': 14, 'TCGA-MESO': 14, 'TCGA-ACC': 14, 'TCGA-UCS': 10, 'TCGA-CHOL': 10, 'TCGA-UVM': 9, 'TCGA-DLBC': 9})
Label distribution (val): Counter({'TCGA-BRCA': 50, 'TCGA-HNSC': 26, 'TCGA-THCA': 26, 'TCGA-KIRC': 25, 'TCGA-LUSC': 23, 'TCGA-LUAD': 23, 'TCGA-LGG': 21, 'TCGA-PRAD': 20, 'TCGA-STAD': 19, 'TCGA-SKCM': 18, 'TCGA-BLCA': 17, 'TCGA-LIHC': 16, 'TCGA-COAD': 13, 'T

In [3]:
import pandas as pd
DATA_PATH = "/Users/arujatiwary/Desktop/DL+IS-2/tcga_preprocessed.csv"
df = pd.read_csv(DATA_PATH)
print(df.shape)

(10459, 17950)
